In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
승하차_파일 = "../../data/결과/승하차/통합/1호선_승하차인원_통합.csv"
혼잡도_파일 = "../../data/결과/혼잡도/통합/1호선_혼잡도_통합.csv"
시간표_파일 = "../../data/결과/운행시간표/통합/1호선_열차운행시각표.csv"

승하차_df = pd.read_csv(승하차_파일, encoding="euc-kr")
혼잡도_df = pd.read_csv(혼잡도_파일, encoding="euc-kr")
시간표_df = pd.read_csv(시간표_파일, encoding="euc-kr")

In [3]:
time_cols = [col for col in 승하차_df.columns if ":" in col]
# 분석 대상 설정
target_station = "서울역"  # 또는 다른 역명
target_daytype = "평일"  # 또는 "주말"

# 승차 데이터 long 형식
승차_long = pd.melt(
    승하차_df[
        (승하차_df["구분"] == "승차")
        & (승하차_df["역명"] == target_station)
        & (승하차_df["평일주말"] == target_daytype)
    ],
    id_vars=["평일주말", "호선", "역번호", "역명"],
    value_vars=time_cols,
    var_name="시간",
    value_name="승차인원",
)

# 하차 데이터 long 형식
하차_long = pd.melt(
    승하차_df[
        (승하차_df["구분"] == "하차")
        & (승하차_df["역명"] == target_station)
        & (승하차_df["평일주말"] == target_daytype)
    ],
    id_vars=["평일주말", "호선", "역번호", "역명"],
    value_vars=time_cols,
    var_name="시간",
    value_name="하차인원",
)

# 혼잡도 데이터 long 형식 (상행만 사용)
혼잡도_long = pd.melt(
    혼잡도_df[
        (혼잡도_df["역명"] == target_station)
        & (혼잡도_df["평일주말"] == target_daytype)
        & (혼잡도_df["구분"] == "상행")
    ],
    id_vars=["평일주말", "호선", "역번호", "역명", "구분"],
    value_vars=time_cols,
    var_name="시간",
    value_name="혼잡도",
)

In [4]:
# 승차 + 하차 데이터 병합
merge_df = pd.merge(
    승차_long, 하차_long, on=["평일주말", "호선", "역번호", "역명", "시간"], how="inner"
)

# 혼잡도 추가 병합 (구분 컬럼 제거)
final_df = pd.merge(
    merge_df,
    혼잡도_long.drop("구분", axis=1),
    on=["평일주말", "호선", "역번호", "역명", "시간"],
    how="left",
)

# 시간순 정렬을 위한 정수형 시간 컬럼 추가
final_df["시간_int"] = final_df["시간"].apply(lambda x: int(x.split(":")[0]))
final_df = final_df.sort_values("시간_int").reset_index(drop=True)

# 결과 미리 확인
final_df.head()

MemoryError: Unable to allocate 5.18 GiB for an array with shape (695581740,) and data type int64

In [ ]:
def get_station_day_df(역명, 평일주말, 승차_long, 하차_long, 혼잡도_long):
    # 미리 필터링
    승차 = 승차_long[(승차_long["역명"] == 역명) & (승차_long["평일주말"] == 평일주말)]
    하차 = 하차_long[(하차_long["역명"] == 역명) & (하차_long["평일주말"] == 평일주말)]
    혼잡도 = 혼잡도_long[
        (혼잡도_long["역명"] == 역명)
        & (혼잡도_long["평일주말"] == 평일주말)
        & (혼잡도_long["구분"] == "상행")
    ]
    # 병합
    merge_df = pd.merge(
        승차, 하차, on=["평일주말", "호선", "역번호", "역명", "시간"], how="inner"
    )
    final_df = pd.merge(
        merge_df,
        혼잡도.drop("구분", axis=1),
        on=["평일주말", "호선", "역번호", "역명", "시간"],
        how="left",
    )
    return final_df


# 사용 예:
# result_df = get_station_day_df('서울역', '평일', 승차_long, 하차_long, 혼잡도_long)

In [ ]:
features = final_df[["승차인원", "하차인원", "혼잡도"]].fillna(0).values
target = final_df["승차인원"].values  # 예시로 승차인원 예측, 필요시 혼잡도 등으로 대체

scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

In [ ]:
def create_sequences(data, target, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        xs.append(data[i : i + seq_length])
        ys.append(target[i + seq_length])
    return np.array(xs), np.array(ys)


SEQ_LEN = 4
X, y = create_sequences(features_scaled, target, SEQ_LEN)

In [ ]:
model = Sequential(
    [LSTM(16, input_shape=(SEQ_LEN, X.shape[2])), Dense(8, activation="relu"), Dense(1)]
)
model.compile(loss="mse", optimizer="adam")
model.fit(X, y, epochs=60, batch_size=4, verbose=1)

In [ ]:
def predict_station_passenger(model, scaler, df, input_time, seq_len=4):
    # 시간 매칭: 입력값 (예: '13:00')에 가장 가까운 시간 결정
    df = df.sort_values("시간_int").reset_index(drop=True)
    # 입력된 시간의 인덱스(정확히 없으면 이전 시간 사용)
    시간_int = int(input_time.split(":")[0])
    idx = df[df["시간_int"] == 시간_int].index
    if len(idx) == 0 or idx[0] < seq_len:
        return "초기 구간은 예측 불가"
    idx = idx[0]
    x_input = df[["승차인원", "하차인원", "혼잡도"]].fillna(0).values
    x_input = scaler.transform(x_input)
    x_input = x_input[idx - seq_len : idx].reshape(1, seq_len, 3)
    pred = model.predict(x_input)
    return f"{input_time} 기준 예상 승차인원: {int(pred[0,0])}명"


# 예시 실행
print(predict_station_passenger(model, scaler, final_df, "7:00"))